In [280]:
import pandas as pd
from lxml import html
import json

import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk

import pymorphy2

## Исходные данные

In [4]:
src_path = '/data/share/lab05data/'

test_ids_lst = [1, 3587, 770, 3593, 3084, 3607, 536, 1561, 2588, 3931, 2079, 2481, 2406, 1580, 2605, 2101, 55, 1081, 62, 63, 3649, 3139, 581, 70, 3143, 1098, 1099, 589, 2134, 88, 2649, 357, 2139, 1117, 358, 1634, 189, 1644, 109, 3184, 3188, 2166, 2682, 3198, 3125, 3203, 1669, 646, 3719, 1560, 144, 3730, 1174, 3223, 3738, 3739, 1183, 3744, 1188, 1701, 711, 680, 535, 2174, 1714, 799, 695, 1720, 2462, 1211, 701, 702, 1215, 630, 194, 3783, 3784, 3279, 1150, 2259, 724, 383, 3800, 2265, 3293, 3296, 740, 1767, 3820, 3822, 3824, 241, 3317, 1270, 2006, 1281, 231, 812, 2311, 2830, 2323, 789, 2010, 409, 133, 1311, 2343, 3368, 2860, 3886, 2864, 307, 2357, 2358, 2954, 2362, 3901, 1343, 822, 1345, 1347, 324, 3639, 2377, 312, 3405, 2896, 3300, 3418, 859, 861, 352, 3426, 356, 3429, 2918, 658, 2924, 1909, 2423, 1344, 1407, 2436, 1928, 1418, 3566, 3481, 1428, 406, 1432, 2457, 2288, 2461, 1691, 2977, 2467, 1956, 2981, 2982, 2984, 1961, 939, 429, 2992, 433, 3067, 2036, 2485, 444, 1469, 3920, 3524, 455, 504, 2028, 3025, 3538, 2517, 3542, 2521, 869, 3547, 336, 3038, 2531, 998, 2535, 2025, 1704, 2029, 2030, 510, 1521, 2548, 1525, 870, 836, 1024, 2731, 2646]
base_ids_lst = [10,11,12,13,14,15,16,17,18,19,1,20,2,3,4,5,6,7,8,9]

## Соберем датафреймы из файлов

In [237]:
text_lst = []

for idx in base_ids_lst:
    with open(src_path + 'base_'+ str(idx) +'.txt') as f:
        text = f.read()
    text_lst.append([idx, text, 'base'])

for idx in test_ids_lst:
    with open(src_path + 'test_'+ str(idx) +'.txt') as f:
        text = f.read()
    text_lst.append([idx, text, 'test'])    
    
df = pd.DataFrame(text_lst, columns=['id','text','group'])

In [238]:
df

,id,text,group
0,10,<p>Компания Wargaming в лице команды-разработч...,base
1,11,<p>JavaScript Developer (Worldoftanks.ru)</p> ...,base
2,12,<p> </p> <p>Компания Wargaming приглашает в св...,base
3,13,<p> </p> <p>Компания Wargaming приглашает в св...,base
4,14,<p>Разработчик нашей мечты (java developer)</p...,base
...,...,...,...
215,870,<p><strong>Обязанности</strong>:</p> <ul> <li>...,test
216,836,<p><strong>Обязанности</strong>:</p> <ul> <li>...,test
217,1024,<p><strong>Обязанности: </strong></p> <ul> <li...,test
218,2731,<p><strong>WELLNESS CLUB EVO ПРИГЛАШАЕТ НА РАБ...,test


## Очистим от HTML

In [239]:
df['text_clean'] = df.text.apply(lambda x: html.fromstring(x).text_content())

In [240]:
df

,id,text,group,text_clean
0,10,<p>Компания Wargaming в лице команды-разработч...,base,Компания Wargaming в лице команды-разработчико...
1,11,<p>JavaScript Developer (Worldoftanks.ru)</p> ...,base,JavaScript Developer (Worldoftanks.ru) Компа...
2,12,<p> </p> <p>Компания Wargaming приглашает в св...,base,Компания Wargaming приглашает в свои ряды ед...
3,13,<p> </p> <p>Компания Wargaming приглашает в св...,base,Компания Wargaming приглашает в свои ряды ед...
4,14,<p>Разработчик нашей мечты (java developer)</p...,base,Разработчик нашей мечты (java developer) Ищем ...
...,...,...,...,...
215,870,<p><strong>Обязанности</strong>:</p> <ul> <li>...,test,"Обязанности: выполнение плана личных продаж,к..."
216,836,<p><strong>Обязанности</strong>:</p> <ul> <li>...,test,"Обязанности: работа на кассе, ведение журнала..."
217,1024,<p><strong>Обязанности: </strong></p> <ul> <li...,test,Обязанности: сопровождение существующей клие...
218,2731,<p><strong>WELLNESS CLUB EVO ПРИГЛАШАЕТ НА РАБ...,test,WELLNESS CLUB EVO ПРИГЛАШАЕТ НА РАБОТУ ВЕДУЩЕГ...


## Токенизируем и векторизируем тексты

In [ ]:
stopwords = stopwords.words('russian')

In [241]:
m = pymorphy2.MorphAnalyzer()

In [242]:
# франкенштейн собранный из двух лекций и StackOverflow
def token_r(text):
    words = nltk.word_tokenize(text.lower(), language='russian')
    words = [word for word in words if word not in string.punctuation]
    return [m.parse(x)[0].normal_form for x in words]

In [243]:
cv = CountVectorizer(tokenizer=token_r, 
                     stop_words=stopwords,
                     ngram_range=(1, 2))

In [244]:
matrix = cv.fit_transform(df['text_clean'].values).toarray()

In [245]:
matrix.shape

(220, 23059)

In [246]:
# Убедимся что у нас все красиво
cv.vocabulary_

{'компания': 9838,
 'wargaming': 4107,
 'лицо': 10712,
 'команда-разработчик': 9748,
 'танковый': 20411,
 'портал': 14885,
 'world': 4258,
 'of': 2921,
 'tanks': 3826,
 'worldoftanks.ru': 4264,
 'рада': 17153,
 'пригласить': 15430,
 'свой': 18669,
 'ряд': 18471,
 'ещё': 7647,
 'успешный': 21677,
 'нацелить': 11915,
 'результат': 17913,
 'победа': 14292,
 'javascript': 2384,
 'developer': 1618,
 'готовый': 6757,
 'поручить': 14903,
 'разработка': 17370,
 'сопровождение': 19545,
 'javascript-код': 2416,
 'сайт': 18520,
 'например': 11808,
 'соблюдение': 19323,
 'принять': 15674,
 'стилистика': 20024,
 'код': 9486,
 'участие': 21756,
 'code': 1426,
 'review': 3434,
 'поиск': 14523,
 'решение': 18061,
 'улучшение': 21356,
 'кода/архитектура': 9556,
 'общий': 12703,
 'js-решение': 2510,
 'рамка': 17676,
 'несколько': 12093,
 'проект': 16131,
 'требование': 20979,
 'уверенный': 21223,
 'знание': 8259,
 'vanilla': 4071,
 'css/dom/html5': 1534,
 'верстать': 5713,
 'прийтись': 15475,
 'опыт': 1

## Рассчитаем косинусные расстояния

In [247]:
cos_m = cosine_similarity(matrix, matrix[0:20])

In [248]:
cos_m.shape

(220, 20)

In [252]:
# Загоним матрицу в df
df_cos = pd.DataFrame(cos_m)

# Сразу посчитаем сумму растояний для каждой строки
df_cos['sum'] = df_cos.sum(axis=1)

# Объединим с основным датафреймом
df_result = df.merge(df_cos, how='left', left_index=True, right_index=True)

In [253]:
df_result

,id,text,group,text_clean,0,1,2,3,4,5,...,11,12,13,14,15,16,17,18,19,sum
0,10,<p>Компания Wargaming в лице команды-разработч...,base,Компания Wargaming в лице команды-разработчико...,1.000000,0.992345,0.317391,0.317391,0.185449,0.202319,...,0.236414,0.141799,0.020896,0.262789,0.019960,0.121414,0.021450,1.000000,1.000000,6.623229
1,11,<p>JavaScript Developer (Worldoftanks.ru)</p> ...,base,JavaScript Developer (Worldoftanks.ru) Компа...,0.992345,1.000000,0.314961,0.314961,0.182457,0.197558,...,0.230850,0.138463,0.024114,0.256604,0.025987,0.118557,0.025134,0.992345,0.992345,6.571853
2,12,<p> </p> <p>Компания Wargaming приглашает в св...,base,Компания Wargaming приглашает в свои ряды ед...,0.317391,0.314961,1.000000,1.000000,0.236778,0.219285,...,0.256156,0.180024,0.016279,0.295473,0.010558,0.140740,0.016339,0.317391,0.317391,5.603350
3,13,<p> </p> <p>Компания Wargaming приглашает в св...,base,Компания Wargaming приглашает в свои ряды ед...,0.317391,0.314961,1.000000,1.000000,0.236778,0.219285,...,0.256156,0.180024,0.016279,0.295473,0.010558,0.140740,0.016339,0.317391,0.317391,5.603350
4,14,<p>Разработчик нашей мечты (java developer)</p...,base,Разработчик нашей мечты (java developer) Ищем ...,0.185449,0.182457,0.236778,0.236778,1.000000,0.195265,...,0.178910,0.158331,0.010833,0.237684,0.013797,0.125886,0.012232,0.185449,0.185449,3.883347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,870,<p><strong>Обязанности</strong>:</p> <ul> <li>...,test,"Обязанности: выполнение плана личных продаж,к...",0.083941,0.081966,0.116815,0.116815,0.090380,0.056410,...,0.097118,0.095293,0.000000,0.109905,0.000000,0.100621,0.000000,0.083941,0.083941,1.499472
216,836,<p><strong>Обязанности</strong>:</p> <ul> <li>...,test,"Обязанности: работа на кассе, ведение журнала...",0.108438,0.105886,0.225863,0.225863,0.098378,0.113036,...,0.156304,0.127299,0.000000,0.171290,0.000000,0.085811,0.000000,0.108438,0.108438,2.183157
217,1024,<p><strong>Обязанности: </strong></p> <ul> <li...,test,Обязанности: сопровождение существующей клие...,0.052320,0.051089,0.074727,0.074727,0.081371,0.078951,...,0.057709,0.063175,0.003667,0.107945,0.004282,0.142681,0.000000,0.052320,0.052320,1.144695
218,2731,<p><strong>WELLNESS CLUB EVO ПРИГЛАШАЕТ НА РАБ...,test,WELLNESS CLUB EVO ПРИГЛАШАЕТ НА РАБОТУ ВЕДУЩЕГ...,0.180585,0.176335,0.227287,0.227287,0.200149,0.364325,...,0.172751,0.139241,0.000000,0.282644,0.005096,0.123319,0.003286,0.180585,0.180585,3.304239


## Средняя близость к эталонным текстам

In [261]:
mean_sum = df_result.query('group == "test"')['sum'].mean()
mean_sum

2.4047065131542014

In [292]:
defined_lst = []
other_lst = []

for item in df_result.query('group == "test"')[['id','sum']].values:
    if item[1] > mean_sum:
        defined_lst.append(int(item[0]))
    else:
        other_lst.append(int(item[0]))

In [312]:
result_dic = {
    "defined": sorted(defined_lst),
    "other": sorted(other_lst)
}

In [314]:
with open('../lab05.json', 'w') as outfile:
    json.dump(result_dic, outfile)